<a href="https://www.kaggle.com/code/prasaddash/prasad-brats-d1?scriptVersionId=92048628" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
import os
import random
import torch
import pandas as pd
from tqdm.notebook import tqdm_notebook
from torchvision import transforms, utils
import cv2

In [2]:
import tarfile
file = tarfile.open('../input/brats-2021-task1/BraTS2021_Training_Data.tar')

file.extractall('./brain_images')
file.close()

In [ ]:
PATIENT_ID='00558'
T1_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1.nii.gz"
T1CE_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1ce.nii.gz"
T2_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t2.nii.gz"
FLAIR_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_flair.nii.gz"
MASK_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_seg.nii.gz"
img_t1=nib.load(T1_PATH)
img_t1ce=nib.load(T1CE_PATH)
img_t2=nib.load(T2_PATH)
img_flair=nib.load(FLAIR_PATH)
img_mask=nib.load(MASK_PATH)
image_data_t1=img_t1.get_fdata()
image_data_t1ce=img_t1ce.get_fdata()
image_data_t2=img_t2.get_fdata()
image_data_flair=img_flair.get_fdata()
mask=img_mask.get_fdata()
print(img_t1.header)
zero_matrix=np.zeros((240,240))
start=-1
end=1e9
nonEmptyMasks=[]
for i in range(0,155):
    if not np.array_equal(zero_matrix,mask[:,:,i]):
        if start==-1:
                start=i
        nonEmptyMasks.append(i)
        if i<end:
            end=i

def one_normal(x):
    if x==0:
        return 0
    else:
        return 1
print(image_data_t1.min())
plt.rcParams.update({'text.color': "red"})
LAYER=random.choice(nonEmptyMasks)
LAYER=93
fig = plt.figure(figsize=(100,50))
rows = 5
cols=1
fig.add_subplot(rows, cols, 1)
cmask=np.vectorize(one_normal)(mask[:,:,LAYER])
plt.imshow(image_data_t1[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T1")
fig.add_subplot(rows, cols,2)
plt.imshow(image_data_t1ce[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T1CE")
fig.add_subplot(rows, cols, 3)
plt.imshow(image_data_t2[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T2")
fig.add_subplot(rows, cols, 4)
plt.imshow(image_data_flair[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("FLAIR")
fig.add_subplot(rows, cols, 5)
plt.imshow(mask[:,:,LAYER],cmap='gray')
plt.axis('off')
plt.title("MASK")
fig.suptitle(f'Layer Number {LAYER}', fontsize=20)

In [ ]:
MAX_T1=-1
MIN_T1=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T1=max(MAX_T1,image_data.max())
    print(i,image_data.min())
    MIN_T1=min(MIN_T1,image_data.min())
print("Max Intensity in T1 Images : ",MAX_T1)
print("Min Intensity in T1 Images : ",MIN_T1)
MAX_T1CE=-1
MIN_T1CE=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1CE'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1ce.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T1CE=max(MAX_T1CE,image_data.max())
    MIN_T1CE=min(MIN_T1CE,image_data.min())
print("Max Intensity in T1 Images : ",MAX_T1CE)
print("Min Intensity in T1 Images : ",MIN_T1CE)
MAX_T2=-1
MIN_T2=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T2'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t2.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T2=max(MAX_T2,image_data.max())
    MIN_T2=min(MIN_T2,image_data.min())
print("Max Intensity in T2 Images : ",MAX_T2)
print("Min Intensity in T2 Images : ",MIN_T2)
MAX_FLAIR=-1
MIN_FLAIR=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='FLAIR'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_flair.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_FLAIR=max(MAX_FLAIR,image_data.max())
    MIN_FLAIR=min(MIN_FLAIR,image_data.min())
print("Max Intensity in FLAIR Images : ",MAX_FLAIR)
print("Min Intensity in FLAIR Images : ",MIN_FLAIR)

In [3]:
labels=pd.read_csv('../input/mgmt-labels/train_labels_MGMT.csv')
CURRENT_FOLDER='./brain_images'
ALL_PATIENTS=[]
for f in tqdm_notebook(os.listdir(CURRENT_FOLDER),desc='Progress'):
    path_to_dir=os.path.join(CURRENT_FOLDER,f)
    if os.path.isdir(path_to_dir):
        ALL_PATIENTS.append(f)
ALL_MGMT_PATIENTS=[str(label).zfill(5) for label in labels['BraTS21ID']]
print("Total Number of patients : ",len(ALL_PATIENTS))
print("Total Number of patients with MGMT Data : ",len(ALL_MGMT_PATIENTS))
print("Total Number of patients with MGMT Methylation Positive : ",len(labels[labels['MGMT_value']==1]))
print("Total Number of patients with MGMT Methylation Negative : ",len(labels[labels['MGMT_value']==0]))
for i in ALL_MGMT_PATIENTS:
    if f'BraTS2021_{i}' not in ALL_PATIENTS:
        print(i)
ALL_MGMT_PATIENTS=[i for i in ALL_MGMT_PATIENTS if f'BraTS2021_{i}' in ALL_PATIENTS]

Progress:   0%|          | 0/1252 [00:00<?, ?it/s]

Total Number of patients :  1251
Total Number of patients with MGMT Data :  585
Total Number of patients with MGMT Methylation Positive :  307
Total Number of patients with MGMT Methylation Negative :  278
00169
00197
00245
00308
00408
00564
00794
00998


In [4]:
NEGATIVE_IMAGES=0
POSITIVE_IMAGES=0
ALL_DATA=[]
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='Progress'):
    zero_matrix=np.zeros((240,240))
    start=-1
    end=1e9
    nonEmptyMasks=[]
    MASK_PATH=f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_seg.nii.gz"
    img_mask=nib.load(MASK_PATH)
    mask=img_mask.get_fdata()
    for j in range(0,155):
        if (mask[:,:,j]>0).astype('float64').sum()>300 :
            ALL_DATA.append((i,j,labels[labels['BraTS21ID']==int(i)]['MGMT_value'].values[0]))
            if(labels[labels['BraTS21ID']==int(i)]['MGMT_value'].values[0]):
                POSITIVE_IMAGES+=1
            else:
                NEGATIVE_IMAGES+=1

Progress:   0%|          | 0/577 [00:00<?, ?it/s]

In [5]:
print("Final No. of Images in the dataset:",len(ALL_DATA))
print("No of Images labelled true",POSITIVE_IMAGES)
print("No of Images labelled false",NEGATIVE_IMAGES)

Final No. of Images in the dataset: 30976
No of Images labelled true 16387
No of Images labelled false 14589


In [6]:
import pandas as pd
dataset=pd.DataFrame(ALL_DATA,columns=['id','slice','label'])

In [ ]:
dataset.tail()

In [7]:
def ExtractRegion(img):
    img_x,img_y=img.shape
    for i in range(img_x):
        if img[i,:].sum()>0:
            ul=i
            break
    for i in range(img_x-1,0,-1):
        if img[i,:].sum()>0:
            dl=i
            break
    for i in range(img_y):
        if img[:,i].sum()>0:
            ll=i
            break
    for i in range(img_y-1,0,-1):
        if img[:,i].sum()>0:
            rl=i
            break
    img = img[ul:dl+1,ll:rl+1]
    img =  cv2.resize(img, (240,240), interpolation = cv2.INTER_CUBIC)
    img = cv2.cvtColor(img.astype('float32'),cv2.COLOR_GRAY2RGB)
    return img

In [56]:
from sklearn import tree
from sklearn.metrics import classification_report
def predict(test,model):
    labels = []
    X=[]
    for i in tqdm_notebook(range(int(len(test)))):
        img = nib.load(f"./brain_images/BraTS2021_{dataset['id'].loc[i]}/BraTS2021_{dataset['id'].loc[i]}_t1.nii.gz").get_fdata()
        mask = nib.load(f"./brain_images/BraTS2021_{dataset['id'].loc[i]}/BraTS2021_{dataset['id'].loc[i]}_seg.nii.gz").get_fdata()
        labels.append(dataset['label'].loc[i])
        img = img[:,:,dataset['slice'].loc[i]]
        mask = mask[:,:,dataset['slice'].loc[i]]
        mask = (mask>0).astype('float64')
        img = img*mask
        img = ExtractRegion(img)
        img = img.reshape(3,240,240)
        X.append(img)
    X = torch.tensor(X)
    labels = np.array(labels)
    preds_vgg = model_vgg(X)
    preds_resnet = model_resnet(X)
    X = torch.cat((preds_vgg,preds_resnet),dim=1)
    y_preds = model.predict(X)
    print(classification_report(labels,y_preds))
    

def fit(train_set):
    model = tree.DecisionTreeClassifier()
    labels = []
    X=[]
    for i in tqdm_notebook(range(int(len(train_set)))):
        img = nib.load(f"./brain_images/BraTS2021_{dataset['id'].loc[i]}/BraTS2021_{dataset['id'].loc[i]}_t1.nii.gz").get_fdata()
        mask = nib.load(f"./brain_images/BraTS2021_{dataset['id'].loc[i]}/BraTS2021_{dataset['id'].loc[i]}_seg.nii.gz").get_fdata()
        labels.append(dataset['label'].loc[i])
        img = img[:,:,dataset['slice'].loc[i]]
        mask = mask[:,:,dataset['slice'].loc[i]]
        mask = (mask>0).astype('float64')
        img = img*mask
        img = ExtractRegion(img)
        img = img.reshape(3,240,240)
        X.append(img)
    X=torch.tensor(X)
    labels = np.array(labels)
    preds_vgg = model_vgg(X)
    preds_resnet = model_resnet(X)
    X = torch.cat((preds_vgg,preds_resnet),dim=1)
    model.fit(X,labels)
    print(labels)
    return model


In [ ]:
m=fit(dataset.loc[0:230])
predict(dataset.loc[500:1000],m)

  0%|          | 0/231 [00:00<?, ?it/s]

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1]


  0%|          | 0/501 [00:00<?, ?it/s]

In [8]:
dataset

,id,slice,label
0,00000,51,1
1,00000,52,1
2,00000,53,1
3,00000,54,1
4,00000,55,1
...,...,...,...
30971,01010,71,0
30972,01010,72,0
30973,01010,73,0
30974,01010,74,0


In [ ]:
features = {}
for i in tqdm_notebook(range(len(dataset))):
    img = nib.load(f"./brain_images/BraTS2021_{dataset['id'].loc[i]}/BraTS2021_{dataset['id'].loc[i]}_t1.nii.gz").get_fdata()
    mask = nib.load(f"./brain_images/BraTS2021_{dataset['id'].loc[i]}/BraTS2021_{dataset['id'].loc[i]}_seg.nii.gz").get_fdata()
    img = img[:,:,dataset['slice'].loc[i]]
    mask = mask[:,:,dataset['slice'].loc[i]]
    mask = (mask>0).astype('float64')
    img = img*mask
    img = ExtractRegion(img)
    img = img.reshape(3,240,240)
    img = torch.tensor([img])
    preds_vgg = model_vgg(img)
    preds_resnet = model_resnet(img)
    preds_combined = torch.cat((preds_vgg,preds_resnet),dim=1)

In [19]:
import torchvision.models as models
import torch.nn as nn
model_vgg=models.vgg16(pretrained=True)
model_resnet=models.resnet18(pretrained=True)
model_vgg.classifier = model_vgg.classifier[:2]
model_resnet.fc = nn.Identity()
for param in model_vgg.parameters():
    param.requires_grad = False
for param in model_resnet.parameters():
    param.requires_grad = False
# num_ftrs_resnet = model_resnet.fc.in_features
# num_ftrs_vgg=model_vgg.classifier[0].in_features
# model_resnet.fc=nn.Linear(num_ftrs_resnet,256)
# model_vgg.classifier=nn.Linear(num_ftrs_vgg,256)
# def computeFeautures(img):
#     img = img.reshape((3,240,240))
#     img = np.array([img])
#     img = torch.tensor(img)
#     vgg_out = model_vgg(img)
#     resnet_out = model_resnet(img)
#     comb_out=torch.cat((vgg_out,resnet_out),dim=1)
#     return comb_out

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
model_vgg

In [ ]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook
model_vgg.AdaptiveAvgPool.register_forward_hook(get_features('feats'))

In [ ]:
dataset

In [ ]:
from torch.utils.data import Dataset
import cv2
class MRI_Dataset(Dataset):
    def __init__(self,dataset, annotations_file, transform=None):
        self.dataset=dataset
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        _id,_slice=self.dataset[idx]
        img_path = f"./brain_images/BraTS2021_{str(_id).zfill(5)}/BraTS2021_{_id}_t1.nii.gz"
        image = nib.load(img_path)
        image = image.get_fdata()[:,:,_slice]
        image = cv2.cvtColor(image.astype('float32'),cv2.COLOR_GRAY2RGB)
        label = self.img_labels[self.img_labels['BraTS21ID']==int(_id)]['MGMT_value'].values[0]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
training_data=MRI_Dataset(ALL_DATA,'../input/mgmt-labels/train_labels_MGMT.csv')

In [ ]:
class Normalise_range(object):
    def __init__(self, min_t,max_t):
        self.min_t = min_t
        self.max_t = max_t

    def __call__(self, sample):
        image = sample
        ## Minmax Normalisation to 0 to 255
        image=image-self.min_t
        image=image/(self.max_t-self.min_t)
        return image


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torchvision.models as models
train,test = train_test_split(ALL_DATA, test_size=0.33, random_state=42)
training_data=MRI_Dataset(train,'../input/mgmt-labels/train_labels_MGMT.csv',transform=transforms.Compose([
                                               Normalise_range(MIN_T1,MAX_T1)  ,
                                               transforms.ToTensor() ,
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                           ]))
test_data=MRI_Dataset(test,'../input/mgmt-labels/train_labels_MGMT.csv')
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
training_data[32]

In [ ]:
import torch.nn as nn

In [ ]:
out = model_resnet(i.float())

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg=torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True).float()
        self.resnet=torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True).float()
        for param in self.vgg.parameters():
            param.requires_grad = False
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_ftrs_resnet = self.resnet.fc.in_features
        num_ftrs_vgg=self.vgg.classifier[0].in_features
        self.resnet.fc=nn.Linear(num_ftrs_resnet,256)
        self.vgg.classifier=nn.Linear(num_ftrs_vgg,256)
        self.fc=nn.Sequential(
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,1),
            nn.Sigmoid()
        )
        
    def forward(self, img):
        vgg_out = self.vgg(img)
        resnet_out = self.resnet(img)
        comb_out=torch.cat((vgg_out,resnet_out),dim=1)
        result=self.fc(comb_out)
        print(vgg_out.shape)
        print(resnet_out.shape)
        print(comb_out.shape)
        print(result.shape)
        print(result)
        

In [ ]:
Model=NeuralNetwork()

In [ ]:
for i,j in train_dataloader:
    Model(i.float())

In [ ]:
print(model_vgg_inftrs)
print(model_vgg)

In [ ]:
print(model_resnet)